<a href="https://colab.research.google.com/github/22104071/GS_071/blob/main/TwitterSentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install kaggle

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
100% 80.9M/80.9M [00:03<00:00, 33.4MB/s]
100% 80.9M/80.9M [00:03<00:00, 24.6MB/s]


In [ ]:
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import warnings
warnings.filterwarnings('ignore')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [5]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [36]:
# Load data
df = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1', header=None)

# Rename columns
df.columns = ['target', 'id', 'date', 'flag', 'user', 'text']

In [37]:
df.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [38]:
df.describe()

,target,id
count,1.600000e+06,1.600000e+06
mean,2.000000e+00,1.998818e+09
std,2.000001e+00,1.935761e+08
min,0.000000e+00,1.467810e+09
25%,0.000000e+00,1.956916e+09
50%,2.000000e+00,2.002102e+09
75%,4.000000e+00,2.177059e+09
max,4.000000e+00,2.329206e+09


In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   target  1600000 non-null  int64 
 1   id      1600000 non-null  int64 
 2   date    1600000 non-null  object
 3   flag    1600000 non-null  object
 4   user    1600000 non-null  object
 5   text    1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


In [40]:
df.shape

(1600000, 6)

In [41]:
df.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [42]:
df=df.dropna(axis=0,how='any')

In [43]:
df.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [44]:
df.shape


(1600000, 6)

In [45]:
df['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

Convert the target 4 to 1

In [46]:
df.replace({'target':{4:1}},inplace=True)

In [47]:
df['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

In [48]:
df.replace({'target':{4:1}},inplace=True)

In [49]:
df['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

0,0 -> Negative Tweet


1,4 -> Positive Tweet

Stemming

It is the process of reducing a word to its root

example: acting,actor,actress -> act

In [50]:
ps = PorterStemmer()

In [51]:
def stemming(text):
  stemmed_content = re.sub('[^a-zA-Z]',' ',text)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [ps.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [53]:
df['stemmed_content'] = df['text'].apply(stemming)

In [54]:
print(df['target'])


0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [59]:
# Seperating the data and label
x = df['stemmed_content'].values
y = df['target'].values

In [60]:
print(x)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [61]:
print(y)

[0 0 0 ... 1 1 1]


Splitting the dataset into training data and test data

In [62]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,stratify=y,random_state=2)


In [63]:
print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

(1280000,) (320000,) (1280000,) (320000,)


In [64]:
print(x_train)

['watch saw iv drink lil wine' 'hatermagazin'
 'even though favourit drink think vodka coke wipe mind time think im gonna find new drink'
 ... 'eager monday afternoon'
 'hope everyon mother great day wait hear guy store tomorrow'
 'love wake folger bad voic deeper']


In [65]:
print(x_test)

['mmangen fine much time chat twitter hubbi back summer amp tend domin free time'
 'ah may show w ruth kim amp geoffrey sanhueza'
 'ishatara mayb bay area thang dammit' ...
 'destini nevertheless hooray member wonder safe trip' 'feel well'
 'supersandro thank']


In [66]:
print(y_train)

[1 1 0 ... 1 1 0]


In [67]:
print(y_test)

[0 1 0 ... 1 0 1]


In [68]:
#Converting the textual data into numerical data

In [69]:
vectorizer = TfidfVectorizer()
x_train_features = vectorizer.fit_transform(x_train)
x_test_features = vectorizer.transform(x_test)

In [70]:
print(x_train)

['watch saw iv drink lil wine' 'hatermagazin'
 'even though favourit drink think vodka coke wipe mind time think im gonna find new drink'
 ... 'eager monday afternoon'
 'hope everyon mother great day wait hear guy store tomorrow'
 'love wake folger bad voic deeper']


In [71]:
print(x_test)

['mmangen fine much time chat twitter hubbi back summer amp tend domin free time'
 'ah may show w ruth kim amp geoffrey sanhueza'
 'ishatara mayb bay area thang dammit' ...
 'destini nevertheless hooray member wonder safe trip' 'feel well'
 'supersandro thank']


Training the Machine
Learning Model

Logistic Regression

In [72]:
model = LogisticRegression(max_iter=1000)

In [73]:
model.fit(x_train_features,y_train)

LogisticRegression(max_iter=1000)

Model Evaluation

Accuracy Score

In [74]:
#Accuracy Score Calculation

In [75]:
x_train_prediction = model.predict(x_train_features)
training_data_accuracy = accuracy_score(x_train_prediction,y_train)


In [76]:
print('Accuracy Score of the training data:',training_data_accuracy)

Accuracy Score of the training data: 0.81018984375


In [77]:
x_test_prediction = model.predict(x_test_features)
testing_data_accuracy = accuracy_score(x_test_prediction,y_test)

In [83]:
print('Accuracy Score of the testing data:',testing_data_accuracy*100,'%')

Accuracy Score of the testing data: 77.80375000000001 %


Saving the Trained Model

In [79]:
import pickle

In [80]:
filename = 'trained_model.sav'
pickle.dump(model,open(filename,'wb'))

Using the saved model for future predictions

In [81]:
#Loading the saved model

In [85]:
loaded_model = pickle.load(open('trained_model.sav','rb'))
x_new = x_test[200]
print(y_test[200])

x_new_vectorized = vectorizer.transform([x_new])  # Transform the text into a vector

prediction = loaded_model.predict(x_new_vectorized)  # Predict using the vectorized input
print(prediction)
if (prediction[0]==0):
  print('The tweet is Negative')
else:
  print('The tweet is Positive')

1
[1]
The tweet is Positive
